In [20]:
import jsonlines
import torchaudio
from datasets import Dataset
from transformers import WhisperTokenizer, WhisperForConditionalGeneration, WhisperProcessor, WhisperFeatureExtractor, AutoModelForSequenceClassification, EarlyStoppingCallback, TrainerCallback
from pathlib import Path
import torch, random
import librosa, os
from dotenv import load_dotenv
import evaluate
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from datasets import load_metric
import jiwer, re
import inflect
from torch.utils.data import DataLoader


In [21]:
# augment = Compose([
#     AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.1),
#     TimeStretch(min_rate=0.8, max_rate=1.25, p=0.1),
#     PitchShift(min_semitones=-4, max_semitones=4, p=0.1),
#     Shift(p=0.1)
# ])

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = "openai/whisper-small.en"
feature_extractor = WhisperFeatureExtractor.from_pretrained(model)
tokenizer = WhisperTokenizer.from_pretrained(model, language="en", task="transcribe")
processor = WhisperProcessor.from_pretrained(model, language="en", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained(model)


model.to(device)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

In [23]:
load_dotenv()

TEAM_NAME = os.getenv("TEAM_NAME", "7up")
TEAM_TRACK = os.getenv("TEAM_TRACK", "advanced")


input_dir = Path(f"/home/jupyter/{TEAM_TRACK}")
# input_dir = Path(f"../../data/{TEAM_TRACK}/train")
results_dir = Path(f"/home/jupyter/{TEAM_NAME}")
# results_dir = Path("results")
results_dir.mkdir(parents=True, exist_ok=True)
instances = []

data = {'key': [], 'audio': [], 'transcript': []}
with jsonlines.open(input_dir / "asr.jsonl") as reader:
    for obj in reader:
        # if len(data['key']) < 3200:
        for key, value in obj.items():
            data[key].append(value)


In [5]:
# input_str = data['transcript'][0]
# labels = tokenizer(input_str).input_ids
# decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
# decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

# print(f"Input:                 {input_str}")
# print(f"Decoded w/ special:    {decoded_with_special}")
# print(f"Decoded w/out special: {decoded_str}")
# print(f"Are equal:             {input_str == decoded_str}")

In [6]:
# p = inflect.engine()
# def load_audio(audio_path):
#     speech_array, sampling_rate = torchaudio.load(audio_path)
#     return speech_array.squeeze().numpy(), sampling_rate

# def normalize_text(text):
#     # Replace numbers with words
#     def replace_with_words(match):
#         number = match.group(0)
#         return ' '.join(p.number_to_words(digit) for digit in number)
    
#     # Use regex to find numbers and replace them with words
#     text = re.sub(r'\b\d+\b', replace_with_words, text)
#     return text


In [7]:
dataset = Dataset.from_dict(data)
dataframe = dataset.to_pandas()
# Shuffle the dataset
dataset = dataset.shuffle(seed=24)
# dataframe = dataframe.sample(frac=1, random_state=42)

train_size = int(0.9* len(dataset))
val_size = int(0.1 * len(dataset))
# test_size = len(dataset) - train_size - val_size

train_dataset = dataset.select(range(train_size))
val_dataset = dataset.select(range(train_size, train_size + val_size))
# test_dataset = dataset.select(range(train_size + val_size, train_size + val_size + test_size))

In [8]:
# input_str = train_dataset[0]["transcript"]
# labels = tokenizer(input_str).input_ids
# decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
# decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

# print(f"Input:                 {input_str}")
# print(f"Decoded w/ special:    {decoded_with_special}")
# print(f"Decoded w/out special: {decoded_str}")
# print(f"Are equal:             {input_str == decoded_str}")

In [9]:
# train_dataset

In [10]:
# hypotheses = []
# for audio_path, transcript in zip(data['audio'], data['transcript']):
#     # Load and preprocess audio
#     audio_input, sampling_rate = load_audio(input_dir / "audio" / audio_path)
    
#     # Ensure the audio is at the expected sample rate
#     if sampling_rate != processor.feature_extractor.sampling_rate:
#         resampler = torchaudio.transforms.Resample(sampling_rate, processor.feature_extractor.sampling_rate)
#         audio_input = resampler(torch.tensor(audio_input))

#     # Process the audio input
#     inputs = processor(audio_input, sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt")

#     # Generate transcription
#     with torch.no_grad():
#         generated_ids = model.generate(inputs["input_features"])

#     # Decode the transcription
#     transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
#     transcription = normalize_text(transcription)
#     hypotheses.append(transcription)
#     print(transcription)

# # Calculate WER
# wer = jiwer.wer(data['transcript'], hypotheses)
# print(f"Word Error Rate (WER): {wer}") #0.14893617021276595

# # medium.en Word Error Rate (WER): 0.255 quite fast like 2-3 mins for 10
# # whisper large v2 Word Error Rate (WER): 0.09 like 6-7 mins for 10
# # distil whisper large v3 Word Error Rate (WER): 0.165 like 2-3 mins for 10
# # distil whisper large v2 Word Error Rate (WER): 0.17 like 2 mins for 10 (using this)

In [11]:
# def preprocess_data(examples):
#     input_values = []
#     attention_masks = []
#     labels = []

#     for audio_path, transcript in zip(examples['audio'], examples['transcript']):
#         speech_array, sampling_rate = torchaudio.load(input_dir / "audio" / audio_path)
#         processed = processor(speech_array.squeeze(0), sampling_rate=sampling_rate, return_tensors="pt", padding=True)
        
#         # Process labels with the same processor settings
#         # with processor.as_target_processor():
#         transcript = transcript.translate(str.maketrans('', '', string.punctuation))
#         label = tokenizer(transcript, return_tensors="pt", padding=True)
                
#         input_values.append(processed.input_features.squeeze(0))
#         # Create attention masks based on the input values
#         attention_mask = torch.ones_like(processed.input_features)
#         attention_mask[processed.input_features == processor.tokenizer.pad_token_id] = 0  # Set padding tokens to 0
#         attention_masks.append(attention_mask.squeeze(0))

#         # Ensure labels are padded to the same length as inputs if needed
#         padded_label = torch.full((processed.input_features.shape[1],), -100, dtype=torch.int64)
#         actual_length = label.input_ids.shape[1]
#         padded_label[:actual_length] = label.input_ids.squeeze(0)
#         labels.append(padded_label)

#     # Concatenate all batches
#     examples['input_values'] = torch.stack(input_values)
#     examples['attention_mask'] = torch.stack(attention_masks)
#     examples['labels'] = torch.stack(labels)

#     return examples

In [12]:
# def prepare_dataset(examples):
#     for audio_path, transcript in zip(examples['audio'], examples['transcript']):
#         audio, sr = librosa.load(input_dir / "audio" / audio_path, sr=16000)

#         # Compute log-Mel input features from input audio array 
#         input_features = feature_extractor(audio, sampling_rate=sr).input_features[0]

#         # Encode target text to label ids 
#         labels = tokenizer(transcript).input_ids

#         return {
#         "input_features": input_features,
#         "labels": labels
#         }

def prepare_dataset(example):
    audio_path = input_dir / "audio" / example["audio"]

    # Load audio data
    audio, sr = librosa.load(audio_path, sr=16000)
    
    # augmented_samples = augment(samples=audio, sample_rate=sr)
    # Compute log-Mel input features from input audio array 
    input_features = feature_extractor(audio, sampling_rate=sr)

    # Encode target text to label ids 
    labels = tokenizer(example["transcript"]).input_ids

    return {
        "input_features": input_features,
        "labels": labels
    }

In [13]:
processed_train = train_dataset.map(prepare_dataset)
processed_val = val_dataset.map(prepare_dataset)
# processed_test = test_dataset.map(prepare_dataset)
processed_train

Map:   0%|          | 0/3150 [00:00<?, ? examples/s]

Map:   0%|          | 0/350 [00:00<?, ? examples/s]

Dataset({
    features: ['key', 'audio', 'transcript', 'input_features', 'labels'],
    num_rows: 3150
})

In [14]:
# import torch

# from dataclasses import dataclass
# from typing import Any, Dict, List, Union

# @dataclass
# class DataCollatorSpeechSeq2SeqWithPadding:
#     processor: Any
#     decoder_start_token_id: int

#     def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
#         # split inputs and labels since they have to be of different lengths and need different padding methods
#         # first treat the audio inputs by simply returning torch tensors
#         input_features = [{"input_features": feature["input_features"]} for feature in features]
#         batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

#         # get the tokenized label sequences
#         label_features = [{"input_ids": feature["labels"]} for feature in features]
#         # pad the labels to max length
#         labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

#         # replace padding with -100 to ignore loss correctly
#         labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

#         # if bos token is appended in previous tokenization step,
#         # cut bos token here as it's append later anyways
#         if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
#             labels = labels[:, 1:]

#         batch["labels"] = labels

#         return batch


In [15]:
# data_collator = DataCollatorSpeechSeq2SeqWithPadding(
#     processor=processor,
#     decoder_start_token_id=model.config.decoder_start_token_id,
# )

In [16]:
# import evaluate

# metric = evaluate.load("wer")

# def compute_metrics(pred):
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # replace -100 with the pad_token_id
#     label_ids[label_ids == -100] = tokenizer.pad_token_id

#     # we do not want to group tokens when computing the metrics
#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#     wer = 100 * metric.compute(predictions=pred_str, references=label_str)

#     return {"wer": wer}

In [17]:
# from transformers import Seq2SeqTrainingArguments

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./whisper-mdeium-en-7up",  # change to a repo name of your choice
#     per_device_train_batch_size=16,
#     gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
#     learning_rate=1e-5,
#     warmup_steps=500,
#     max_steps=5000,
#     gradient_checkpointing=True,
#     fp16=True,
#     evaluation_strategy="steps",
#     per_device_eval_batch_size=8,
#     predict_with_generate=True,
#     generation_max_length=225,
#     save_steps=1000,
#     eval_steps=1000,
#     logging_steps=25,
#     report_to=["tensorboard"],
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
#     push_to_hub=True,
# )


# from transformers import Seq2SeqTrainer

# trainer = Seq2SeqTrainer(
#     args=training_args,
#     model=model,
#     train_dataset=processed_train,
#     eval_dataset=processed_val,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     tokenizer=processor.feature_extractor,
# )

In [18]:
# trainer.train()


In [24]:
# from datasets import load_metric
# from transformers import Seq2SeqTrainingArguments
# from transformers import Seq2SeqTrainer, Adafactor

# # Load the WER metric
# metric = load_metric("wer")

# def compute_metrics(pred):
#     pred_ids = pred.predictions
#     label_ids = pred.label_ids

#     # Flatten the predictions and labels
#     if isinstance(pred_ids, tuple):
#         pred_ids = pred_ids[0]
#     pred_ids = pred_ids.argmax(axis=-1) if pred_ids.ndim > 2 else pred_ids

#     # Replace -100 with pad token id in label_ids
#     label_ids[label_ids == -100] = tokenizer.pad_token_id

#     # Decode the predictions and labels
#     pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
#     label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

#     # Compute the Word Error Rate (WER)
#     wer = metric.compute(predictions=pred_str, references=label_str)
    
#     return {"wer": wer}

# class DataCollatorSpeechSeq2SeqWithPadding:
#     def __init__(self, processor):
#         self.processor = processor

#     def __call__(self, features):
#         # Separate input_features and labels
#         input_features = [feature["input_features"] for feature in features]
#         labels = [feature["labels"] for feature in features]

#         # Pad input_features and labels
#         batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

#         # Ensure input_features have the correct shape
#         batch["input_features"] = batch["input_features"].squeeze(1)  # Remove the extra dimension if necessary

#         # Pad labels
#         labels_batch = self.processor.tokenizer.pad({"input_ids": labels}, return_tensors="pt", padding=True)

#         # Replace -100 in the labels as we can't compute loss with -100 values
#         labels_batch["input_ids"][labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id] = -100

#         # Add the labels to the batch
#         batch["labels"] = labels_batch["input_ids"]

#         return batch

# # Create the data collator instance
# data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor)

# # Initialize the Trainer

# training_args = Seq2SeqTrainingArguments(
#     output_dir="./whisper-small-en-7up-v5",  # change to a repo name of your choice
#     per_device_train_batch_size=2,
#     gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
#     learning_rate=0.000003,
#     # warmup_steps=200,
#     max_steps=480,
#     gradient_checkpointing=True,
#     fp16=True,
#     evaluation_strategy="steps",
#     per_device_eval_batch_size=2,
#     generation_max_length=225,
#     save_steps=40,
#     eval_steps=40,
#     logging_steps=100,
#     report_to=["tensorboard"],
#     load_best_model_at_end=True,
#     metric_for_best_model="wer",
#     greater_is_better=False,
#     push_to_hub=True
# )

# trainer = Seq2SeqTrainer(
#     model=model,
#     args=training_args,
#     train_dataset=processed_train,
#     eval_dataset=processed_val,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
#     tokenizer=processor.feature_extractor,
# )

# # Train the model
# trainer.train()

In [ ]:
# torch.cuda.empty_cache()

In [2]:
from transformers import WhisperFeatureExtractor
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import json, re, torch, torchaudio, inflect, string
from io import BytesIO

model = WhisperForConditionalGeneration.from_pretrained("cadzchua/whisper-small-en-7up-v2")
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small.en")
processor = WhisperProcessor.from_pretrained("openai/whisper-small.en", language="en", task="transcribe")
model = WhisperForConditionalGeneration.from_pretrained("cadzchua/whisper-small-en-7up")
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')
p = inflect.engine()

def transcribe(audio_bytes: bytes) -> str:
    # Load and preprocess audio
    audio_input, sampling_rate = load_audio(audio_bytes)

    # Ensure the audio is at the expected sample rate
    if sampling_rate != processor.feature_extractor.sampling_rate:
        resampler = torchaudio.transforms.Resample(sampling_rate, processor.feature_extractor.sampling_rate)
        audio_input = resampler(torch.tensor(audio_input))

    # Process the audio input
    inputs = processor(audio_input, sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt")
    inputs = {key: value.to('cuda' if torch.cuda.is_available() else 'cpu') for key, value in inputs.items()}
    
    # Generate transcription
    with torch.no_grad():
        generated_ids = model.generate(inputs["input_features"])

    # Decode the transcription
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # normalized_text = normalize_text(transcription)

    return transcription

def load_audio(audio_path):
    audio_file = BytesIO(audio_path)
    speech_array, sampling_rate = torchaudio.load(audio_file)
    return speech_array.squeeze().numpy(), sampling_rate

def normalize_text(text):
    
    # Replace numbers with words
    def replace_with_words(match):
        number = match.group(0)
        return ' '.join(p.number_to_words(digit) for digit in number)

    # Use regex to find numbers and replace them with words
    text = re.sub(r'\b\d+\b', replace_with_words, text)

    text = (text.translate(str.maketrans('', '', string.punctuation)).lower()).strip()
    # standby, onto is supposed to be 2 word
    #risky = whiskey
    #cnonvert number to words
    return text


with open("/home/jupyter/advanced/asr.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        audio_file = data['audio']
        expected_transcript = data['transcript']
        with open("/home/jupyter/advanced/audio/" + audio_file, 'rb') as audio_f:
            audio_bytes = audio_f.read()
        result = transcribe(audio_bytes)
        result = (result.translate(str.maketrans('', '', string.punctuation)).lower()).strip()
        expected_transcript = (expected_transcript.translate(str.maketrans('', '', string.punctuation)).lower()).strip()
        # expected_transcript = normalize_text(expected_transcript)
        if (result != expected_transcript):
            print(f"Predicted: {result}\nActual: {expected_transcript}\n{result == expected_transcript}\n\n")

Predicted: alpha echo mike papa deploy emp tool heading zero eight five engage purple red and silver fighter jet
Actual: alfa echo mike papa deploy emp tool heading zero eight five engage purple red and silver fighter jet
False


Predicted: turret alpha deploy machine gun on yellow grey and orange helicopter heading two two five
Actual: turret alfa deploy machine gun on yellow grey and orange helicopter heading two two five
False


Predicted: control your prepare to deploy emp on a white yellow and silver commercial aircraft heading zero eight five target locked engage emp now standby for confirmation
Actual: control here prepare to deploy emp on a white yellow and silver commercial aircraft heading zero eight five target locked engage emp now stand by for confirmation
False


Predicted: turret alpha engage the grey orange and yellow fighter jet at zero five five with surfacetoair missiles turret bravo standby for further orders
Actual: turret alpha engage the grey orange and yellow fi